# Step5 演習01 MNISTの多クラス分類
---

本演習では、手書き数字のMNISTをRNNを用いて分類します。

**はじめに**
- for文やwhile文の利用は明示的な利用指示がない場所での利用は避けてください。

**本演習の目的**
- RNNをNumPyのみで実装する

## ライブラリのインストール

まずはじめに、本演習で利用するライブラリのインポートを行います。

- [numpy](http://www.numpy.org) 数値計算を行うための基本パッケージの公式ドキュメント
- [matplotlib](http://matplotlib.org) グラフ描画ライブラリの基本パッケージの公式ドキュメント
- [tensorflow](https://www.tensorflow.org/) 機械学習用のライブラリの公式ドキュメント


`%matplotlib inline` はnotebook上で使える[magic function](http://ipython.readthedocs.io/en/stable/interactive/magics.html)の一つで、これによりmatplotlibをインタラクティブに使うことできます。

In [1]:
import numpy as np
import time
%matplotlib inline 
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

## データセット
まずMNISTデータ・セットを読み込みます。

In [2]:
# データの読み込み
mnist = input_data.read_data_sets("MNIST_data/", 
                                  one_hot=True, 
                                  validation_size=0)

# データのシャッフル
permutation = np.random.permutation(mnist.train._images.shape[0])
mnist.train._images = mnist.train.images[permutation]
mnist.train._labels = mnist.train.labels[permutation]

#データの正規化
mean = np.mean(mnist.train.images)
std = np.std(mnist.train.images)
mnist.train._images = (mnist.train.images-mean)/std
mnist.test._images = (mnist.test.images-mean)/std

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


この演習ではMNISTの画像を行毎にシーケンスデータとしてRNNにいれます。（数字の８の例です）

<img src="./img/step5_mnist_sequence.png" width="620" height="300">

In [3]:
%%HTML #HTMLをセル内で書くことのできるようにするmagic functionです
<img src="img/step5_mnist.gif" width="240" height="240">

## モデルの構築

### モデルパラメータの準備

今回は下記のモデルパラメータを用います。隠れ層は100付近の値に設定します。

In [4]:
hidden_size = 100 # 隠れ層のサイズ
image_size = 28 # 入力画像のサイズ
class_num = 10 #　分類するカテゴリ数（0~9）

### モデルの重みの初期化

**【課題１】**　RNNのパラメータと勾配そしてAdamを使用する際に必要なパラメータを初期化します。

ここではモデルのパラメータをCNNのときと同様に[glorot_uniform](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)で初期化します。バイアスはゼロで初期化します。numpyでの一様分布生成には[np.random.uniform](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.uniform.html)を用います。また、[np.zeros](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.zeros.html)を用いてゼロの配列を生成します。


["glorot_uniform"](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
$$\pm\sqrt{\frac{6}{(input\_channel+ output\_channel)}}$$

この演習では[Adam](https://arxiv.org/pdf/1412.6980.pdf)を使ってRNNを学習させてみます。それにあたり、いくつかの配列を用意します。これらは重みと同じ形をしたゼロの配列で初期化します。これには[np.zeros_like](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros_like.html)を用います。

`init_parameters`の最後では重みのパラメータとAdamに必要なパラメータをまとめて返します。

In [5]:
#Coursedele-02 Step5 QuestionNumber1 3edc123b66020b7fbe6053158fdead12
def init_parameters(image_size, hidden_size, class_num):
    ###############START CODE HERE###############
    #入力から隠れ層の重み（入力サイズ、隠れ層のユニット数）
    std1 = np.sqrt(6 / (image_size + hidden_size))
    U = np.random.uniform(low = -std1, high = +std1, size = (image_size, hidden_size))

    #隠れ層から隠れ層の重み（隠れ層のユニット数、隠れ層のユニット数）
    std2 = np.sqrt(6 / (hidden_size + hidden_size))
    W = np.random.uniform(low = -std2, high = +std2, size = (hidden_size, hidden_size))

    #隠れ層から出力層の重み（隠れ層のユニット数、クラス数）
    std3 = np.sqrt(6 / (hidden_size + class_num))
    V = np.random.uniform(low = -std3, high = std3, size = (hidden_size, class_num))

    # 隠れ層のバイアス（1, 隠れ層のユニット数）
    b = np.zeros((1, hidden_size))
    # 出力層のバイアス（1, クラス数）
    c = np.zeros((1, class_num))

    # Adamを使用するにあたって必要なパラメータの初期化
    mU, mW, mV = np.zeros_like(U), np.zeros_like(W), np.zeros_like(V)
    mb, mc =  np.zeros_like(b), np.zeros_like(c)
    vU, vW, vV =np.zeros_like(U), np.zeros_like(W), np.zeros_like(V)
    vb, vc = np.zeros_like(b), np.zeros_like(c)
    
    ################END CODE HERE################
    
    # パラメータをタプルにまとめます。
    params = (U, W, V, b, c)
    ms = (mU, mW, mV, mb, mc)
    vs = (vU, vW, vV, vb, vc)
    return params, ms, vs

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [6]:
%%bash
./validation_client.py dele-02 5 1 Step5_01.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### フォワードプロパゲーションの実装

**【課題２】**　ここではRNNの順伝播を実装します。

RNNのフォワードプロパゲーションは教材のスライドでは以下のように書きました。


** フォワードプロパゲーション（ミニバッチの次元がないとき） **
$$
\begin{eqnarray*}
\boldsymbol{a}{(t)}&=&\boldsymbol{b}+\boldsymbol{Wh}{(t-1)}+\boldsymbol{Ux}{(t)}\\
\boldsymbol{h}{(t)}&=&tanh(\boldsymbol{a}{(t)})\\
\boldsymbol{o}{(t)}&=&\boldsymbol{c}+\boldsymbol{Vh}{(t)}\\
\boldsymbol{\hat{y}}{(t)}&=&softmax(\boldsymbol{o}{(t)})
\end{eqnarray*}
$$

しかしながら、実装する際は少し書き方を以下のように改める必用があります。これは、$\boldsymbol{x}$にはバッチの次元があり上記通りに実装すると計算ができないからです。例えば$\boldsymbol{Ux}^{(t)}$を計算するときに$\boldsymbol{U}=(image\_size, hidden\_size)$,  $\boldsymbol{x}=(batch\_size, image\_size)$となっているので計算ができません。（numpyのブロードキャスト機能を使いたい）なので、以下のように式を変更します。

** フォワードプロパゲーション（ミニバッチの次元があるとき） **
$$
\begin{eqnarray}
\boldsymbol{a}{(t)}&=&\boldsymbol{b}+\boldsymbol{h}{(t-1)}\boldsymbol{W}+\boldsymbol{x}{(t)}\boldsymbol{U}\\
\boldsymbol{h}{(t)}&=&tanh(\boldsymbol{a}{(t)})\\
\boldsymbol{o}{(t)}&=&\boldsymbol{c}+\boldsymbol{h}{(t)}\boldsymbol{V}\\
\boldsymbol{\hat{y}}{(t)}&=&softmax(\boldsymbol{o}{(t)})
\end{eqnarray}
$$

`forward_propagation`には4つの引数があります。
- `X`: 入力画像（バッチサイズ、縦×横）
- `params`: `init_parameters`で用意した重みとバイアス
- `batch_size`: バッチサイズ
- `hidden_size`: 隠れ層のサイズ

この関数内では、
- `params`から重みとバイアスを取り出します。
- `h`: 隠れ層の値（上の式でいう$\boldsymbol{h}$）を記録するために空の辞書を用意します。
- `h[-1]`: フォワードプロパゲーションの１ステップに必要な値（ゼロ）をいれます。
- 入力`X`の形を（バッチサイズ、縦×横）から（バッチサイズ、縦、横）に変更します。
- forループで画像の行数分（２８）回します。ループ内では毎回、入力画像の1行分を（$\boldsymbol{x}^{(t)}$）としていれます。ここではまだ`X`は画像全体なので注意して扱ってください。ループさせる次元は入力画像の行成分なので、初めのループではX[:,0]となります。
- `y_pred`: `o`にソフトマックス関数を適用します。

In [155]:
#Coursedele-02 Step5 QuestionNumber2 6604f8fa48941ea591de1f55d5308a70
def softmax(x, axis=1):
    exp_x = np.exp(x)
    y = exp_x / np.sum(exp_x, axis=axis, keepdims=True) 
    return y

def forward_propagation(X, params, batch_size, hidden_size):
    U, W, V, b, c = params
    h = {}
    h[-1] = np.zeros((batch_size, hidden_size))
    X = X.reshape(-1, 28, 28)
    ###############START CODE HERE###############
    for t in range(28):
        a = b + np.dot(h[t-1], W) + np.dot(X[:, t], U)
        h[t] = np.tanh(a)
    o = c + np.dot(h[t], V)
    # ソフトマックス関数
    y_pred =softmax(o)
    ################END CODE HERE################
    return y_pred, X, h

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [156]:
%%bash
./validation_client.py dele-02 5 2 Step5_01.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### コスト関数の計算

**【課題３】**　ここではコスト関数を実装します。


コスト関数には交差エントロピーを使います。
$$
cost = - \frac{1}{m}\sum_x y(x) \log{\hat{y}(x)}
$$

In [27]:
#Coursedele-02 Step5 QuestionNumber3 758ca7378619b92e6251f1cabf54ebf7
def compute_cost(y_pred, targets, batch_size):
    ###############START CODE HERE###############
    cost = -np.sum(np.log(y_pred)*targets)/float(batch_size)
    ################END CODE HERE################
    return cost

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [28]:
%%bash
./validation_client.py dele-02 5 3 Step5_01.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### numpyの仕様

numpyには少し変わった仕様があります。通常forループで配列の要素（これもまたnp.ndarray)を取り出しそれを変更してももとの配列は変更されません(`test1`の例)。しかしながら、`test2`の例 で`j += 1`のように書くと元の配列の中身が変更されます。注意しなければならないのはこれはforループで取り出される要素がnp.ndarrayの場合です。なので`test3`の例では値は変更されていません。

In [31]:
test1 = np.array([[1,2]])
test2 = np.array([[1,2]])
test3 = np.array([1,2])
print('test1{}'.format(test1))
print('test2{}'.format(test2))
print('test3{}'.format(test3))
for i, j, k in zip(test1, test2, test3):
    i = i + 1
    j += 1
    k += 1
print('test1{}'.format(test1))
print('test2{}'.format(test2))
print('test3{}'.format(test3))

test1[[1 2]]
test2[[1 2]]
test3[1 2]
test1[[1 2]]
test2[[2 3]]
test3[1 2]


### バックプロパゲーションの実装

**【課題４】**　ここでは通時的誤差逆伝播法（back-propagation through time, BPTT)を実装します。

この演習で実装するのは以下のようなRNNで出力は一番最後にしかありません。
<img src="./img/step5_bptt.png" width="360" height="120">


`backward_propagation`は5つの引数があります。
- `params`: `init_parameters`で用意した重みとバイアス
- `X`: 入力画像（バッチサイズ、縦×横）
- `h`: フォワード時の隠れ状態
- `y_pred`:  フォワード時の出力結果
- `targets`: 正解ラベル

この関数内では、
- `params`から重みとバイアスを取り出します。
- 勾配を入れるための配列を作成します。

- forループで画像の行数分（２８）回します。ここでは図で言う右上（出力）から伝播させて行くので`reversed`させます。これにより[0, 1, ..., 27]が[27, ..., 1, 0]となります。
- パラメータ`V`と`c`があるのはt==27の時のみなので、ここでまず、$\frac{\partial C}{\partial \boldsymbol{o}}$を求めます。この演習ではsoftmaxと交差エントロピーを使っているので単純に予測ラベルー正解ラベルになります。

ここでは時刻tがあるので少しややこしいとは思いますが、ニューラルネットワークの時の計算を参考にすれば書けるはずです。**注意**RNNではコスト関数を最後、データ長（seq_length, この演習の場合28）で割る必要があります。よって、逆伝播時も`delta_o`をデータ長割ります。

最後に勾配爆発が起こらないように勾配クリッピングをします。今回は勾配が-1から1の範囲の値を取るように制限します。

In [8]:
#Coursedele-02 Step5 QuestionNumber4 ffa1f4b4e9e6d0f0436cdeae38d22256
def backward_propagation(params, X, h, y_pred, targets):
    #パラメータの取り出し
    U, W, V, b, c = params
    batch_size, seq_length = X.shape[:2]
    # 勾配を入れるための配列の作成
    delta_U, delta_W, delta_V = np.zeros_like(U), np.zeros_like(W), np.zeros_like(V)
    delta_b, delta_c = np.zeros_like(b), np.zeros_like(c)
    delta_h_next = np.zeros_like(h[0]) # h(t-1)の勾配
    for t in reversed(range(28)):
        ###############START CODE HERE###############
        if t == 27:
            # ソフトマックスと交差エントロピーの逆伝播, dC/do
            delta_o = (y_pred - targets) / float(batch_size)
            delta_o /= seq_length
            # dC/dv = do/dV.T * dC/do
            delta_V += np.dot(h[t].T, delta_o)
            # dC/dc = do/dc * dC/do = dC/do, バッチ次元で足し合わせる
            delta_c += np.sum(delta_o, axis = 0)
            # dC/dh = dC/do * do/dh.T
            delta_h = np.dot(delta_o, V.T)
        else:
            delta_h = delta_h_next
        # dC/da = dh/da * dC/dh(t), tanh(x)の勾配は1-tanh^2(x)
        delta_a = (1 - h[t]*h[t]) * delta_h
        # dC/db = da/db * dC/da = dC/da
        delta_b += np.sum(delta_a, axis = 0)
        # dC/dU = da/dU.T * dC/da
        delta_U += np.dot(X[:,t].T, delta_a)
        # dC/dW = da/dW.T * dC/da
        delta_W += np.dot(h[t-1].T, delta_a)
        # dC/dh_next = dC/da * da/dh(t-1).T
        delta_h_next =np.dot(delta_a, W.T)
        ################END CODE HERE################
    grads = (delta_U, delta_W, delta_V, delta_b, delta_c)
    # 勾配クリッピング
    for g in grads:
        np.clip(g, -1, 1, out=g)
    return grads

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [9]:
%%bash
./validation_client.py dele-02 5 4 Step5_01.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### パラメータの更新

**【課題５】**　ADAMを実装します。

ここでは[ADAM](https://arxiv.org/pdf/1412.6980.pdf)でパラメータを更新します。
$$
\begin{eqnarray*}
\boldsymbol{m}_t &=& \beta_1 \boldsymbol{m}_{(t-1)} + (1-\beta_1)\boldsymbol{g}_t\\
\boldsymbol{v}_t &=& \beta_2 \boldsymbol{v}_{(t-1)} + (1-\beta_2)\boldsymbol{g}_t^2\\
\boldsymbol{\hat{m}}_t &=& \frac{\boldsymbol{m}_t}{(1-\beta_1^t)}\\
\boldsymbol{\hat{v}}_t &=& \frac{\boldsymbol{v}_t}{(1-\beta_2^t)}\\
\boldsymbol{p}_t &=& \boldsymbol{p}_{(t-1)} - \alpha \frac{\boldsymbol{\hat{m}_t}}{\sqrt{\boldsymbol{\hat{v}}_t}+\epsilon}
\end{eqnarray*}
$$

`update_parameters`は5つの引数があります。
- `params`: 重みとバイアス
- `grads`: 勾配
- `ms`, `vs`: ADAMを使って必要な値
- `iteration`: 現在のステップ数 `t`（ADAMのbias correction）
- `num`: ここでは、len(params)=5をforループで回している。（paramsにはU, W, V, b, cの5つのパラメータがタプルで格納されている。）例：params[0] はU、grads[0]はUの勾配。

In [12]:
#Coursedele-02 Step5 QuestionNumber5 feb5efc7f144a5a1dba1a0490b7c683c
def update_parameters(params, grads, ms, vs, iteration):
    alpha = 0.001
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-8
    for num in range(len(params)):
        p, g, m, v = params[num], grads[num], ms[num], vs[num]
        ###############START CODE HERE###############
        # Adam
        m_t = beta1 * m + (1 - beta1) * g
        v_t = beta2 * v + (1 - beta2) * g * g
        m_t_hat = m_t / (1 - np.power(beta1, iteration))
        v_t_hat = v_t / (1 - np.power(beta2, iteration))
        params[num][:] = p - alpha * m_t_hat / (np.sqrt(v_t_hat) + eps)
        ################END CODE HERE################
        ms[num][:] = m_t
        vs[num][:] = v_t
    return params

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [13]:
%%bash
./validation_client.py dele-02 5 5 Step5_01.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



## RNNの学習

### RNNの学習
作成したRNNをデータセットで学習させます。本来であれば数十エポック学習させるのですが、時間がかかってしまうのでここでは5エポック学習させます。これでも十分な精度がでるはずです。

In [157]:
%%time
# バッチサイズ
batch_size = 250
# １エポックあたりのステップ数
total_step_train = int(len(mnist.train.labels)/batch_size)
# パラメータの初期化
params, ms, vs = init_parameters(image_size, hidden_size, class_num)
# ロス/精度を記録するための配列
train_cost = []
train_acc = []
for epoch in range(5): #5エポック学習
    for iteration in range(total_step_train):
        inputs, targets = mnist.train.next_batch(batch_size, fake_data=False)
        # 順伝播
        y_pred, X, h = forward_propagation(inputs, params, batch_size, hidden_size)
        # コスト関数
        _cost = compute_cost(y_pred, targets, batch_size) / X.shape[1]
        # 逆伝播
        grads = backward_propagation(params, X, h, y_pred, targets)
        # パラメータのアップデート
        params = update_parameters(params, grads, ms, vs, total_step_train*epoch+iteration+1)
        
        assert not np.isnan(_cost), 'nan in cost'
        # accuracyとcostを記録
        _acc = np.mean(np.argmax(y_pred, axis=1)==np.argmax(targets, axis=1))
        # ログをプリント
        if iteration % 10 == 0:
            print('\r \t iter: {}, cost: {:.3f}, acc: {:.3f}'.format(iteration, _cost, _acc), end='')
        train_cost.append(_cost)
        train_acc.append(_acc)
    # このエポックの平均ロスと精度をプリント
    print('\n{}: epoch, cost: {:.3f}, acc: {:.3f}'.format(epoch, 
                                                          np.mean(train_cost[total_step_train*epoch:]), 
                                                          np.mean(train_acc[total_step_train*epoch:])))

 	 iter: 230, cost: 0.080, acc: 0.160
0: epoch, cost: 0.082, acc: 0.150
 	 iter: 230, cost: 0.060, acc: 0.292
1: epoch, cost: 0.068, acc: 0.270
 	 iter: 230, cost: 0.084, acc: 0.116
2: epoch, cost: 0.071, acc: 0.255
 	 iter: 230, cost: 0.082, acc: 0.108
3: epoch, cost: 0.083, acc: 0.100
 	 iter: 230, cost: 0.082, acc: 0.104
4: epoch, cost: 0.082, acc: 0.104
CPU times: user 1min 51s, sys: 40.6 s, total: 2min 32s
Wall time: 1min 17s


### 学習データの精度とロスのプロット

学習データの精度とロスをプロットしてみます。

In [ ]:
plt.plot(train_cost)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Cost', fontsize=12)

In [ ]:
plt.plot(train_acc)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)

### テストデータの精度とロスの計算

テストデータの精度とロスを計算してみます。

In [ ]:
test_cost = []
test_acc = []
inputs = mnist.test.images
targets = mnist.test.labels
batch_size = len(inputs)
y_pred, x, h = forward_propagation(inputs, params, batch_size, hidden_size)
# コスト関数
_cost = compute_cost(y_pred, targets, batch_size)

_acc = np.mean(np.argmax(y_pred, axis=1)==np.argmax(targets, axis=1))
test_cost.append(_cost)
test_acc.append(_acc)
# ログをプリント
print('cost: {:.3f}, acc: {:.3f}'.format(np.mean(test_cost), np.mean(test_acc)))

最後にモデルの予測とそのときの入力データを確認してみます。セルを何度か実行し、確認してみてください。

In [ ]:
idx = np.random.randint(10000) #　ランダムな数を生成し、idxに格納
print("モデルの予測：", np.argmax(y_pred[idx]))#　idx番目のモデルの予測結果
plt.imshow(inputs.reshape(10000, 28,28)[idx], cmap='gray')#　idx番目の入力データの画像を表示
plt.title("Input Data")
plt.show()